In [7]:
# Modeli eğit ve test et
train_model(model, train_loader, criterion, optimizer, num_epochs)


Epoch [1/5], Loss: 0.2673, Accuracy: 0.9379

Epoch [2/5], Loss: 0.0000, Accuracy: 1.0000

Epoch [3/5], Loss: 0.0000, Accuracy: 1.0000

Epoch [4/5], Loss: 0.0000, Accuracy: 1.0000

Epoch [5/5], Loss: 0.0000, Accuracy: 1.0000


NameError: name 'test_loader' is not defined

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset,random_split

from PIL import Image

In [3]:
batch_size = 64
learning_rate = 0.001
num_epochs = 30
test_split_ratio = 0.2  # Verinin yüzde 20'sini test seti olarak ayır


In [4]:
# Özel Dataset sınıfı oluşturma
class SignatureDataset(Dataset):
    def __init__(self, real_dir, forg_dir, transform=None):
        self.real_dir = real_dir
        self.forg_dir = forg_dir
        self.transform = transform
        # Yalnızca resim dosyalarını ekle (Thumbs.db gibi dosyaları atlar)
        self.real_images = [img for img in os.listdir(real_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.forg_images = [img for img in os.listdir(forg_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.real_images) + len(self.forg_images)

    def __getitem__(self, idx):
        if idx < len(self.real_images):
            img_path = os.path.join(self.real_dir, self.real_images[idx])
            label = 0  # Orijinal imza
        else:
            img_path = os.path.join(self.forg_dir, self.forg_images[idx - len(self.real_images)])
            label = 1  # Sahte imza

        try:
            image = Image.open(img_path).convert("RGB")
        except (UnidentifiedImageError, FileNotFoundError):
            # Hatalı dosyaları atlamak için None döndür
            return None

        if self.transform:
            image = self.transform(image)
        
        return image, label


In [5]:
# Veri dönüşümleri
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset oluşturma
full_dataset = SignatureDataset(
    real_dir="/Users/hbuzun/Documents/GitHub/Signature_recegnator/1_models/1_veri_setleri/1_Cedar_dataset/full_org",
    forg_dir="/Users/hbuzun/Documents/GitHub/Signature_recegnator/1_models/1_veri_setleri/1_Cedar_dataset/full_forg",
    transform=data_transforms
)

# Dataset'leri eğitim ve test setlerine ayırma
test_size = int(len(full_dataset) * test_split_ratio)
train_size = len(full_dataset) - test_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoader oluşturma
def collate_fn(batch):
    # None değerlerini atla
    batch = [item for item in batch if item is not None]
    return torch.utils.data.default_collate(batch)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [6]:
# Modeli yükleme
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




/opt/anaconda3/envs/3-9-13_NN/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/3-9-13_NN/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/hbuzun/.cache/torch/hub/checkpoints/vgg16-397923af.pth
7.7%


KeyboardInterrupt: 

In [7]:
# Eğitim fonksiyonu
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            # None döndüren dosyaları atla
            if images is None or labels is None:
                continue
            
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Test fonksiyonu
def test_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    accuracy = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# Daha basit bir CNN modeli
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        # Özellik çıkarma katmanları
        self.features = nn.Sequential(
            # Blok 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Daha az filtre
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Özellik haritası boyutunu azaltır
            
            # Blok 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Blok 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Sınıflandırıcı katmanları
        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 256),  # Daha az nöron
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(256, num_classes)  # Çıkış sınıf sayısı
        )
    
    def forward(self, x):
        x = self.features(x)  # Özellik çıkarımı
        x = torch.flatten(x, 1)  # Düzleştirme (flatten)
        x = self.classifier(x)  # Sınıflandırıcı
        return x

# Modeli tanımlama
model = SimpleCNN(num_classes=2)  # 2 sınıf (örneğin, imza tanımlama)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Model özet bilgisi (isteğe bağlı)
print(model)

train_model(model, train_loader, criterion, optimizer, num_epochs)

test_model(model, test_loader, criterion)




SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=50176, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=2, bias=True)
  )
)
Epoch [1/30], Loss: 0.9881, Accuracy: 0.5232
Epoch [2/30], Loss: 0.6040, Accuracy: 0.6856
Epoch [3/30], Loss: 0.5233, Accuracy: 0.7491
Epoch [4/30], Loss: 0.3885, Accuracy: 0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Güncellenmiş CNN modeli
class DisplacementFeatureCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(DisplacementFeatureCNN, self).__init__()
        
        # Sınıf özelliği olarak num_classes'i sakla
        self.num_classes = num_classes
        
        # Özellik çıkarma katmanları
        self.features = nn.ModuleList([
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        ])
        
        # MaxPool2d işlemi
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Sınıflandırıcı katmanları
        self.classifier = None  # Daha sonra giriş boyutuna göre tanımlanacak
    
    def forward(self, x):
        displacement_features = []
        for layer in self.features:
            if isinstance(layer, nn.Conv2d):
                x = layer(x)
            elif isinstance(layer, nn.ReLU):
                x = layer(x)
                pooled, indices = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
                
                # Displacement feature: Havuzlama sonrası max değerlerin konumları
                indices_flat = indices.view(indices.size(0), -1).float()
                displacement_features.append(indices_flat)  # Düzleştirilmiş displacement feature'lar
                
                x = pooled

        # Düzleştirme
        x = torch.flatten(x, 1)
        displacement_features = torch.cat(displacement_features, dim=1)
        
        # Özellik haritası ve displacement birleştirilir
        combined_features = torch.cat((x, displacement_features), dim=1)
        
        # Sınıflandırıcıyı dinamik olarak oluştur
        if self.classifier is None:
            input_dim = combined_features.size(1)
            self.classifier = nn.Sequential(
                nn.Linear(input_dim, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.3),
                nn.Linear(256, self.num_classes)  # num_classes sınıf özelliği olarak kullanılıyor
            )
            self.classifier = self.classifier.to(combined_features.device)
        
        # Sınıflandırıcı
        x = self.classifier(combined_features)
        return x





# Model tanımlama
model = DisplacementFeatureCNN(num_classes=2)  # 2 sınıf
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Model özet bilgisi (isteğe bağlı)
print(model)

# Eğitim ve test fonksiyonları tanımlandıktan sonra kullanılabilir
train_model(model, train_loader, criterion, optimizer, num_epochs)
test_model(model, test_loader, criterion)


DisplacementFeatureCNN(
  (features): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Epoch [1/30], Loss: 2602.3014, Accuracy: 0.4915
Epoch [2/30], Loss: 2312.6819, Accuracy: 0.5123
Epoch [3/30], Loss: 1551.1903, Accuracy: 0.5450
Epoch [4/30], Loss: 1531.7122, Accuracy: 0.5421
Epoch [5/30], Loss: 1594.4316, Accuracy: 0.5246
Epoch [6/30], Loss: 1357.1197, Accuracy: 0.5492
Epoch [7/30], Loss: 1408.5149, Accuracy: 0.5208
Epoch [8/30], Loss: 1366.9740, Accuracy: 0.5374
Epoch [9/30], Loss: 1354.7941, Accuracy: 0.5317


training: 68m 10.8s

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# Daha basit bir CNN modeli
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        # Özellik çıkarma katmanları
        self.features = nn.Sequential(
            # Blok 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Daha az filtre
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Özellik haritası boyutunu azaltır
            
            # Blok 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Blok 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Sınıflandırıcı katmanları
        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 256),  # Daha az nöron
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(256, num_classes)  # Çıkış sınıf sayısı
        )
    
    def forward(self, x):
        x = self.features(x)  # Özellik çıkarımı
        x = torch.flatten(x, 1)  # Düzleştirme (flatten)
        x = self.classifier(x)  # Sınıflandırıcı
        return x

# Modeli tanımlama
model = SimpleCNN(num_classes=2)  # 2 sınıf (örneğin, imza tanımlama)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Model özet bilgisi (isteğe bağlı)
print(model)

train_model(model, train_loader, criterion, optimizer, num_epochs)

test_model(model, test_loader, criterion)



SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=100352, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
  )
)
Epoch [1/20], Loss: 1.2075, Accuracy: 0.4962
Epoch [2/20], Loss: 0.6300, Accuracy: 0.6264
Epoch [3/20], Loss: 0.5260, Accuracy: 0.7386
Epoch [4/20], Loss: 0.4052, Accuracy:

In [20]:
# Modeli dosyaya kaydet
torch.save(model.state_dict(), 'signature_model.pth')
print("Model başarıyla kaydedildi.")


Model başarıyla kaydedildi.


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

# Hiperparametreler
batch_size = 32
learning_rate = 0.0001
num_epochs = 5
test_split_ratio = 0.2

# Özel Dataset sınıfı
class SignatureDataset(Dataset):
    def __init__(self, genuine_dir, forgery_dir, transform=None):
        self.genuine_dir = genuine_dir
        self.forgery_dir = forgery_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Genuine klasöründen gerçek imzaları ekle
        for person_id in os.listdir(genuine_dir):
            person_path = os.path.join(genuine_dir, person_id)
            if os.path.isdir(person_path):
                for img_name in sorted(os.listdir(person_path)):
                    img_path = os.path.join(person_path, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(0)  # Gerçek imza için 0

        # Forgery klasöründen sahte imzaları ekle
        for forgery_type in os.listdir(forgery_dir):
            forgery_type_path = os.path.join(forgery_dir, forgery_type)
            if os.path.isdir(forgery_type_path):
                for person_id in os.listdir(forgery_type_path):
                    person_path = os.path.join(forgery_type_path, person_id)
                    if os.path.isdir(person_path):
                        for img_name in sorted(os.listdir(person_path)):
                            img_path = os.path.join(person_path, img_name)
                            self.image_paths.append(img_path)
                            self.labels.append(1)  # Sahte imza için 1

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Veri dönüşümleri
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset oluşturma
full_dataset = SignatureDataset(genuine_dir="/kaggle/input/utsignature-verification/UTSig/Genuine", forgery_dir="/kaggle/input/utsignature-verification/UTSig/Forgery/Skilled", transform=data_transforms)

# Eğitim ve test setlerine bölme
test_size = int(len(full_dataset) * test_split_ratio)
train_size = len(full_dataset) - test_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# DataLoader oluşturma
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Modeli yükleme ve ayarlama
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Eğitim fonksiyonu
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        accuracy = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

# Test fonksiyonu
def test_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    accuracy = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Modeli eğit
train_model(model, train_loader, criterion, optimizer, num_epochs)

# Modeli test et
test_model(model, test_loader, criterion)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 203MB/s]  


Epoch [1/5], Loss: 0.0172, Accuracy: 0.9891
Epoch [2/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [3/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [4/5], Loss: 0.0000, Accuracy: 1.0000
Epoch [5/5], Loss: 0.0000, Accuracy: 1.0000
Test Loss: 0.0000, Test Accuracy: 1.0000
